In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
#convertir texto en un valor a una matrix, cada columna es una palabra
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import unicodedata
import pickle
import re
from sklearn.metrics import classification_report
from sklearn import metrics

In [134]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [135]:
df.sample(10)

,package_name,review,polarity
86,com.linkedin.android,new version makes it harder to use linkedin ...,0
39,com.facebook.katana,app doesn't work after latest upgrade the fac...,0
723,com.opera.mini.native,annoying sometimes why does opera need to rel...,0
415,com.facebook.orca,messenger/facebook integration issue ever sin...,0
93,com.linkedin.android,bad app for a great site linkedin is great bu...,0
189,com.imangi.templerun2,fun diversion good game to take a quick break...,1
544,com.dropbox.android,"best cloud provider love the flexibility, fea...",1
836,com.hamropatro,usefull no others app like this....,1
811,org.mozilla.firefox,"the best android browser fast, well designed,...",1
453,com.whatsapp,very nice app best feature is all friends get...,1


In [136]:
#Eliminamos los caracteres desconocidos
df['review']=df['review'].str.strip()
df['review']=df['review'].str.lower()
df

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offlin...,0
1,com.facebook.katana,"messenger issues ever since the last update, i...",0
2,com.facebook.katana,profile any time my wife or anybody has more t...,0
3,com.facebook.katana,the new features suck for those of us who don'...,0
4,com.facebook.katana,forced reload on uploading pic on replying com...,0
...,...,...,...
886,com.rovio.angrybirds,loved it i loooooooooooooovvved it because it ...,1
887,com.rovio.angrybirds,all time legendary game the birthday party lev...,1
888,com.rovio.angrybirds,ads are way to heavy listen to the bad reviews...,0
889,com.rovio.angrybirds,fun works perfectly well. ads aren't as annoyi...,1


In [137]:
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.normalize('NFKC') #ANALIZA TEXTO NO LATINO Y LO TRATA DE ARREGLAR
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)
df.sample(10)

,package_name,review,polarity
123,com.king.candycrushsaga,either make the moving forward process 3 bars ...,0
555,jabanaki.todo.todoly,uninstalled i downloaded this app thinking it ...,0
634,com.uc.browser.en,nice update really nice update this update is ...,1
695,com.hamrokeyboard,best nepali typing apps 1.you can type any nep...,1
256,com.android.chrome,bookmarking is no good i used to be able to pi...,0
500,com.Slack,very useful thank god i can finally reply fast...,1
754,com.shirantech.kantipur,very useful for watching tv reading saptahikka...,1
412,com.facebook.orca,causes android device to not register touch th...,0
6,com.facebook.katana,major flaws constant updates and always gettin...,0
712,com.opera.mini.native,save image and low performance can not save an...,0


In [138]:
def normalize_string(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else: 
        result=None
    return result

df['review']=df['review'].apply(normalize_string)
df.sample(10)

,package_name,review,polarity
619,com.uc.browser.en,pls fix these bugs : when ever u set the sear...,0
528,com.dropbox.android,just one irritating issue very nice app... exc...,1
538,com.dropbox.android,preview not available i cannot view my ms offi...,0
123,com.king.candycrushsaga,either make the moving forward process 3 bars ...,0
67,com.twitter.android,account creation issue i tried to create an ac...,0
132,com.king.candycrushsaga,every time i play the moon struck game it does...,0
703,com.opera.mini.native,classic browsing at it's best i've recently up...,1
795,org.mozilla.firefox,trust worthy as always have been a long time u...,1
722,com.opera.mini.native,still not cool guys please this is still not c...,0
716,com.opera.mini.native,bug fixes required this version is much better...,0


In [139]:
X=df['review']
y=df['polarity']

In [140]:
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=563, stratify=y)

In [141]:

#genera un vector, tfid lo pasa a una matriz, clf clasificacion con un binomial de bernoulli
text_clf = Pipeline([('vec',CountVectorizer(stop_words='english')),('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf.fit(X_train, y_train)

Pipeline(steps=[('vec', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [142]:
vec.get_feature_names_out()

array(['000', '04', '0x', ..., 'zespole', 'zoom', 'zooming'], dtype=object)

In [143]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

y_pred = text_clf.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.8079532565668402, 0.7533632286995515, 0.7075961641787106, None)

In [144]:
print('Naive Bayes Train Accuracy = ',metrics.accuracy_score(y_train,text_clf.predict(X_train)))
print('Naive Bayes Test Accuracy = ',metrics.accuracy_score(y_test,text_clf.predict(X_test)))

Naive Bayes Train Accuracy =  0.8727544910179641
Naive Bayes Test Accuracy =  0.7533632286995515


In [145]:
n_iter_search = 5
parameters = {'vec__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = RandomizedSearchCV(text_clf, parameters, n_iter = n_iter_search)
gs_clf.fit(X_train, y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('vec',
                                              CountVectorizer(stop_words='english')),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf', MultinomialNB())]),
                   n_iter=5,
                   param_distributions={'clf__alpha': (0.01, 0.001),
                                        'tfidf__use_idf': (True, False),
                                        'vec__ngram_range': [(1, 1), (1, 2)]})

In [146]:
gs_clf.best_params_

{'vec__ngram_range': (1, 1), 'tfidf__use_idf': False, 'clf__alpha': 0.01}

In [147]:
print('Naive Bayes Train Accuracy (grid random search) = ',metrics.accuracy_score(y_train,gs_clf.predict(X_train)))
print('Naive Bayes Test Accuracy (grid random search) = ',metrics.accuracy_score(y_test,gs_clf.predict(X_test)))

Naive Bayes Train Accuracy (grid random search) =  0.9880239520958084
Naive Bayes Test Accuracy (grid random search) =  0.8340807174887892


In [148]:
y_pred_mejor = gs_clf.predict(X_test)

print(classification_report(y_test, y_pred_mejor))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       146
           1       0.87      0.61      0.72        77

    accuracy                           0.83       223
   macro avg       0.85      0.78      0.80       223
weighted avg       0.84      0.83      0.83       223



In [149]:
best_model = gs_clf.best_estimator_
best_model

Pipeline(steps=[('vec', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(alpha=0.01))])

In [152]:
# Save best model

pickle.dump(best_model, open('../models/best_model.pickle', 'wb'))

In [153]:
modelo = pickle.load(open('../models/best_model.pickle', 'rb')) # lo leemos
modelo.predict(X_test) # lo usamos para predecir nueva X_test

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1])